In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
# Read in data
df = pd.read_csv('data/draftstars.csv')
df = df[df['Playing Status'] != 'OUT'].reset_index(drop=True)
df = df.groupby('Name').head(1).reset_index(drop=True) # How to add this constraint ???
df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Create solver
solver = pywraplp.Solver.CreateSolver('CBC')

# Object to store optimisation results/indicies
player_ix = {k: solver.BoolVar('x_' + str(v)) for (k, v) in zip(df.index, df.index)}

In [3]:
def player_count_constraint(solver):
    ct = solver.Constraint(9, 9, 'total_players')
    
    for x in player_ix:
        ct.SetCoefficient(player_ix[x], 1)
        
    return solver

In [4]:
def position_constraint(solver):

    df['idx_ls'] = [[el] for el in df.index]
    pos_dict = df.groupby('position')['idx_ls'].agg('sum')
    pos_dict = dict(zip(list(pos_dict.index), list(pos_dict.values)))
    
    for pos in pos_dict:
        pos_count = 1 if pos == 'C' else 2
        ct = solver.Constraint(pos_count, pos_count, 'player_pos_' + pos)
        
        for ix in pos_dict[pos]:
            ct.SetCoefficient(player_ix[ix], 1)
            
    return solver

In [5]:
def salary_constraint(solver):
    ct = solver.Constraint(0, 100000, 'total_salary')
    
    for ix in player_ix:
        ct.SetCoefficient(player_ix[ix], int(df.iloc[ix]['salary']))
        
    return solver

In [6]:
def set_obj_function(solver, player_ix):
    objective = solver.Objective()
    
    for ix in player_ix:
        objective.SetCoefficient(player_ix[ix], df.iloc[ix]['form'])
        
    objective.SetMaximization()
    solver.Solve()
    
    return solver, player_ix, objective

In [7]:
def add_constraints(solver):

    cnsts_dict = {}
    cnsts_dict['solver'] = solver
    
    for cnst in [el for el in globals().keys() if el.endswith('constraint')]:
        cnsts_dict[cnst] = globals()[cnst]
        solver = eval(cnst + '(solver)', cnsts_dict)
  
    return solver

In [8]:
solver = add_constraints(solver)
solver, player_ix, objective = set_obj_function(solver, player_ix)
df_team = df.iloc[[bool(int(el.solution_value())) for el in player_ix.values()]]
df_team = df_team[['name', 'position', 'team', 'salary', 'form', 'playing_status']]
df_team['position'] = pd.Categorical(df_team['position'], categories=['PG', 'SG', 'PF', 'SF', 'C'], ordered=True)
df_team = df_team.sort_values('position')

print(f'''
    Generated team total PER: {round(df_team['form'].sum(), 2)}. \n 
    Total Team's salary:  ${round(df_team['salary'].sum(), 2)} 
''')

df_team


    Generated team total PER: 262.18. 
 
    Total Team's salary:  $99680 



,name,position,team,salary,form,playing_status
4,Malcolm Brogdon,PG,Portland Trail Blazers,14220,36.58,"EXPECTED STARTER, PLAYER IS INJURED BUT IS A G..."
9,Coby White,PG,Chicago Bulls,15490,42.50,EXPECTED STARTER
8,Anfernee Simons,SG,Portland Trail Blazers,16010,46.92,"EXPECTED STARTER, PLAYER IS INJURED BUT IS A G..."
11,Ayo Dosunmu,SG,Chicago Bulls,10410,28.42,EXPECTED STARTER
15,Jabari Walker,PF,Portland Trail Blazers,11440,25.42,"EXPECTED STARTER, PLAYER IS INJURED BUT IS A G..."
20,Toumani Camara,PF,Portland Trail Blazers,6550,16.42,NaN
10,Matisse Thybulle,SF,Portland Trail Blazers,6550,20.33,NaN
14,Dalen Terry,SF,Chicago Bulls,6000,9.42,NaN
6,Deandre Ayton,C,Portland Trail Blazers,13010,36.17,EXPECTED STARTER
